In [15]:
from ipywidgets import interact
import os
from matplotlib import pyplot as plt
import ffmpeg
import numpy as np
import subprocess
from utils import get_img_frame, output_dir, get_resolution, Frame

In [4]:
input_video = os.path.abspath('football.webm')
input_png = os.path.abspath('grid.jpg')
file_name = os.path.splitext(os.path.basename(input_video))[0]
output_video = os.path.join(output_dir, 'test_mosh_{}.mp4'.format(
    file_name))  # this ensures we won't over-write your original video


probe = ffmpeg.probe(input_video)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = 378


overlay_ffmpeg = ffmpeg.input(input_png, s='{}x{}'.format(width, height))
fps = 25
start_sec = 0
end_sec = 20

In [5]:
video_info

{'index': 0,
 'codec_name': 'vp8',
 'codec_long_name': 'On2 VP8',
 'profile': '0',
 'codec_type': 'video',
 'codec_time_base': '1001/30000',
 'codec_tag_string': '[0][0][0][0]',
 'codec_tag': '0x0000',
 'width': 800,
 'height': 600,
 'coded_width': 800,
 'coded_height': 600,
 'has_b_frames': 0,
 'sample_aspect_ratio': '4:3',
 'display_aspect_ratio': '16:9',
 'pix_fmt': 'yuv420p',
 'level': -99,
 'field_order': 'progressive',
 'refs': 1,
 'r_frame_rate': '30000/1001',
 'avg_frame_rate': '30000/1001',
 'time_base': '1/1000',
 'start_pts': 3,
 'start_time': '0.003000',
 'disposition': {'default': 1,
  'dub': 0,
  'original': 0,
  'comment': 0,
  'lyrics': 0,
  'karaoke': 0,
  'forced': 0,
  'hearing_impaired': 0,
  'visual_impaired': 0,
  'clean_effects': 0,
  'attached_pic': 0,
  'timed_thumbnails': 0},
 'tags': {'language': 'eng'}}

In [48]:
out, err = (
    ffmpeg
    .input('moshed_videos/temp_mosh_11.avi')
    .overlay(overlay_ffmpeg, eof_action='pass')
    .output('pipe:', format='yuv420', pix_fmt='rgb24')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.imshow(video[frame,:,:,:])



Error: ffmpeg error (see stderr output for detail)

In [40]:
def start_ffmpeg_input(in_filename, fps, start_sec, end_sec):
    print('Starting ffmpeg input')
    args = (
        ffmpeg
            .input(in_filename)
            .output('pipe:', format='rawvideo', pix_fmt='yuv420p', 
                   b='5000k', g=fps * (end_sec - start_sec), keyint_min=999999,
                   ss=start_sec, to=end_sec)
            .compile()
    )
    return subprocess.Popen(args, stdout=subprocess.PIPE)


def start_ffmpeg_output(out_filename, fps, width, height):
    print('Starting ffmpeg output')
    args = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='yuv420p', s='{}x{}'.format(width, height))
            .output(out_filename, pix_fmt='yuv420p', r=fps)
            .overwrite_output()
            .compile()
    )
    return subprocess.Popen(args, stdin=subprocess.PIPE)

def read_frame(in_ffmpeg, width, height):
    print('Read frame')
    # Note: RGB24 == 3 bytes per pixel.
    frame_size = int(width * height / 4 * 6)
    in_bytes = in_ffmpeg.stdout.read(frame_size)
    if len(in_bytes) < 8:
        frame = None
    else:
        assert len(in_bytes) == frame_size
        frame = (
            np
                .frombuffer(in_bytes, np.uint8)
        )
    return frame

def write_frame(out_ffmpeg, frame):
    print('Writing frame')
    out_ffmpeg.stdin.write(
        frame.data
            .astype(np.uint8)
            .tobytes()
    )

In [42]:
input_ffmpeg = start_ffmpeg_input(input_video, 30, 0, 25)
output_ffmpeg = start_ffmpeg_output(output_video, 30, width, height)

while True:
    in_frame = read_frame(input_ffmpeg, width, height)
    if in_frame is None:
        print('End of input stream')
        break
    
    in_frame = Frame(in_frame)
    if in_frame.is_key_frame():
        print('Found keyframe!')
    write_frame(output_ffmpeg, in_frame)
    
print('Waiting for ffmpeg process1')
input_ffmpeg.wait()

print('Waiting for ffmpeg process2')
output_ffmpeg.stdin.close()
output_ffmpeg.wait()

print('Done')


Starting ffmpeg input
Starting ffmpeg output
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame


/home/krulvis/PycharmProjects/you-can-datamosh-on-linux/utils.py:105: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self.header == iframe


Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame


Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame


Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
Read frame
Writing frame
